In [14]:
import h5py
import os
from dotenv import load_dotenv
import numpy as np
import rasterio
from rasterio.transform import from_origin
from pyproj import Transformer
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

load_dotenv()

True

In [15]:
new_root = os.getenv("SO2SAT_DIR", ) #'path to so2sat-lzc42 v4'
dataset = "testing" # training, testing, validation
img_fname = f'{dataset}.h5'
aux_fname = f'{dataset}_geo.h5'
PROJECT_CRS = "EPSG:4326"

In [ ]:
h5_file = os.path.join(new_root,img_fname)

with h5py.File(h5_file, 'r') as fid1:
    sen1_shape = fid1['sen1'].shape
    print("sen1_data shape:", sen1_shape)

with h5py.File(h5_file, 'r') as fid1:
    sen1_data = np.array(fid1['sen1'])  # (N, patchSize, patchSize, 8)
    sen2_data = np.array(fid1['sen2'])
    label_data = np.array(fid1['label'])

h5_file_aux = os.path.join(new_root,aux_fname)

with h5py.File(h5_file_aux, 'r') as fid1:
    tfw_new = np.array(fid1['tfw'])
    epsg_new = np.array(fid1['epsg'])
    coord = np.array(fid1['coord'])

_, labels = np.where(label_data == 1)
labels += 1
lcz_classes, lcz_counts = np.unique(labels, return_counts=True)

sen1_data shape: (24188, 32, 32, 8)


In [ ]:
lcz_dict = {
    1: {"name": "Compact High-Rise", "alt_code": "1", "color": "#8c0000"},
    2: {"name": "Compact Mid-Rise", "alt_code": "2", "color": "#d10000"},
    3: {"name": "Compact Low-Rise", "alt_code": "3", "color": "#ff0000"},
    4: {"name": "Open High-Rise", "alt_code": "4", "color": "#bf4d00"},
    5: {"name": "Open Mid-Rise", "alt_code": "5", "color": "#ff6600"},
    6: {"name": "Open Low-Rise", "alt_code": "6", "color": "#ff9955"},
    7: {"name": "Lightweight Low-Rise", "alt_code": "7", "color": "#faee05"},
    8: {"name": "Large Low-Rise", "alt_code": "8", "color": "#bcbcbc"},
    9: {"name": "Sparsely Built", "alt_code": "9", "color": "#ffccaa"},
    10: {"name": "Heavy Industry", "alt_code": "10", "color": "#555555"},
    11: {"name": "Dense Trees", "alt_code": "A", "color": "#006a00"},
    12: {"name": "Scattered Trees", "alt_code": "B", "color": "#00aa00"},
    13: {"name": "Bush, Scrub", "alt_code": "C", "color": "#648525"},
    14: {"name": "Low Plants", "alt_code": "D", "color": "#b9db79"},
    15: {"name": "Bare Rock or Paved", "alt_code": "E", "color": "#000000"},
    16: {"name": "Bare Soil or Sand", "alt_code": "F", "color": "#fbf7ae"},
    17: {"name": "Water", "alt_code": "G", "color": "#6a6aff"},
}

# Extract colors in class order
colors = [lcz_dict[c]["color"] for c in lcz_classes]

# Plot
plt.figure(figsize=(10, 5))
plt.bar(lcz_classes, lcz_counts, color=colors)
plt.xlabel("LCZ Class")
plt.ylabel("Count")
plt.title(f"LCZ Class Distribution in {dataset} set")
plt.xticks(lcz_classes)
plt.tight_layout()
plt.show()

In [ ]:
bbox_lst = []
start_idx = 0

for sample_idx in range(start_idx, label_data.shape[0]):
    
    sample_sen1 = np.transpose(sen1_data[sample_idx - start_idx], (2, 0, 1))
    sample_sen2 = np.transpose(sen2_data[sample_idx - start_idx], (2, 0, 1))
    sample_label = label_data[sample_idx - start_idx]
    origin_x = tfw_new[sample_idx - start_idx, 0]
    pixel_width = tfw_new[sample_idx - start_idx, 1]
    origin_y = tfw_new[sample_idx - start_idx, 3]
    pixel_height = tfw_new[sample_idx - start_idx, 5]
    transform = from_origin(origin_x, origin_y, pixel_width, pixel_height)
    epsg_code = int(epsg_new[sample_idx - start_idx, 0])
    sen1_outfile = os.path.join(new_root, f'{dataset}/sentinel1/sen1_patch_{sample_idx:06d}.tif')
    
    with rasterio.open(
        sen1_outfile, 'w',
        driver='GTiff',
        height=sample_sen1.shape[1],
        width=sample_sen1.shape[2],
        count=sample_sen1.shape[0],
        dtype=sample_sen1.dtype,
        crs=f'EPSG:{epsg_code}',
        transform=transform
    ) as dst:
        dst.write(sample_sen1)
    sen2_outfile = os.path.join(new_root, f'{dataset}/sentinel2/sen2_patch_{sample_idx:06d}.tif')
    
    with rasterio.open(
        sen2_outfile, 'w',
        driver='GTiff',
        height=sample_sen2.shape[1],
        width=sample_sen2.shape[2],
        count=sample_sen2.shape[0],
        dtype=sample_sen2.dtype,
        crs=f'EPSG:{epsg_code}',
        transform=transform
    ) as dst:
        dst.write(sample_sen2)

    height = sample_sen1.shape[1]
    width = sample_sen1.shape[2]
    xmin = origin_x
    xmax = origin_x + width * pixel_width
    ymax = origin_y
    ymin = origin_y + height * pixel_height  # pixel_height usually negative
    # Reproject bbox
    transformer = Transformer.from_crs(
        f'EPSG:{epsg_code}',
        PROJECT_CRS,
        always_xy=True
    )
    x1, y1 = transformer.transform(xmin, ymin)
    x2, y2 = transformer.transform(xmax, ymax)
    patch_bbox = [x1, y1, x2, y2]
    print(f"Saving sample {sample_idx}: {labels[sample_idx]} - {patch_bbox}")
    bbox_lst.append(patch_bbox)

In [ ]:
from shapely.geometry import box
polygon_lst = [box(*bbox) for bbox in bbox_lst]
bbox_gdf = gpd.GeoDataFrame(geometry=polygon_lst, crs=PROJECT_CRS)
bbox_gdf["patch_id"] = bbox_gdf.index
bbox_gdf['dataset'] = dataset
bbox_gdf.to_file(os.path.join(new_root, f'{dataset}/labels.gpkg'), index=False, driver='GPKG')

In [ ]:
bbox_info_lst = []
for dataset in ["training", "validation", "testing"]:
    bbox_info_path = os.path.join(new_root, f'{dataset}/labels.gpkg')

    bbox_info_gdf = gpd.read_file(bbox_info_path)
    bbox_info_gdf
    bbox_info_gdf['dataset'] = dataset
    bbox_info_lst.append(bbox_info_gdf)
bbox_info_gdf = pd.concat(bbox_info_lst)
bbox_info_gdf

,LCZ_class,patch_id,geometry,dataset
0,6,000000,"POLYGON ((72.96642 33.66244, 72.96642 33.65961...",training
1,17,000001,"POLYGON ((-73.78031 40.82308, -73.78031 40.820...",training
2,17,000002,"POLYGON ((-154.65521 -56.32187, -154.65521 -56...",training
3,2,000003,"POLYGON ((31.28295 29.9883, 31.28295 29.98546,...",training
4,8,000004,"POLYGON ((31.29023 29.92614, 31.29023 29.9233,...",training
...,...,...,...,...
24183,14,024183,"POLYGON ((-70.5682 -33.67462, -70.5682 -33.677...",testing
24184,8,024184,"POLYGON ((113.16117 23.09897, 113.16117 23.096...",testing
24185,3,024185,"POLYGON ((-122.37642 37.58754, -122.37642 37.5...",testing
24186,17,024186,"POLYGON ((-122.33301 37.75676, -122.33301 37.7...",testing


In [11]:
bbox_rxr_gdf = gpd.read_file(os.path.join(new_root, 'patches_reference_rxr.gpkg'))
bbox_rxr_gdf['LCZ_class'] = bbox_info_gdf['LCZ_class'].tolist()
bbox_rxr_gdf['patch_id'] = bbox_rxr_gdf['patch_id'].apply(lambda x: str(f"{x:06d}"))
bbox_rxr_gdf.to_file(os.path.join(new_root, 'patches_reference_rxr.gpkg'), index=False, driver='GPKG')
bbox_rxr_gdf

,patch_id,dataset,geometry,LCZ_class
0,000000,training,"POLYGON ((72.96642 33.65667, 72.96642 33.65961...",6
1,000001,training,"POLYGON ((-73.78031 40.81731, -73.78031 40.820...",17
2,000002,training,"POLYGON ((-154.65521 -56.31613, -154.65521 -56...",17
3,000003,training,"POLYGON ((31.28295 29.98253, 31.28295 29.98546...",2
4,000004,training,"POLYGON ((31.29023 29.92037, 31.29023 29.9233,...",8
...,...,...,...,...
400668,024183,testing,"POLYGON ((-70.5682 -33.68039, -70.5682 -33.677...",14
400669,024184,testing,"POLYGON ((113.16117 23.09319, 113.16117 23.096...",8
400670,024185,testing,"POLYGON ((-122.37642 37.58178, -122.37642 37.5...",3
400671,024186,testing,"POLYGON ((-122.33301 37.75099, -122.33301 37.7...",17


ValueError: You are trying to merge on int64 and str columns for key 'patch_id'. If you wish to proceed you should use pd.concat